In [190]:
# Imports
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException, WebDriverException

import random
import time
import pandas as pd

In [203]:
companies = ['pwc', 'belfius', 'ey', 'bnp paribas', 'john cockerill', 
             'uliege','odoo', 'alter domus', 'nrb', 'cbc', 'kpmg',
             'total energies', 'kbc', 'bdo', 'deloitte', 'ethias']

In [22]:
#companies = ['pwc', 'belfius']

In [167]:
def setup_driver_to_reviews_search(url):
    driver = webdriver.Chrome('/Users/corentin/Documents/chromedriver_2')
    time.sleep(random.uniform(2, 4))
    driver.set_window_size(1920, 1080)
    driver.implicitly_wait(20)
    driver.get('https://www.glassdoor.com/Reviews/index.htm')
    return driver

In [4]:
def get_rid_overlay(driver):
    """
    Sometimes the site shows an overlay that cannot be removed by clicking, which blocks interaction with the site,
    this is some code to remove it.
    """
    driver.execute_script("""
        javascript:(function(){
          document.getElementsByClassName('hardsellOverlay')[0].remove();
          document.getElementsByTagName("body")[0].style.overflow = "scroll";
          let style = document.createElement('style');
          style.innerHTML = `
            #LoginModal {
              display: none!important;
            }
          `;
          document.head.appendChild(style);
          window.addEventListener("scroll", function (event) {
            event.stopPropagation();
          }, true);
        })();
        """)

In [50]:
# Only belgian reviews
def get_belgian_reviews(driver):
    time.sleep(random.uniform(2, 4))
    filter_button = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[2]/div[2]/button/span')
    filter_button.click()
    time.sleep(random.uniform(2, 4))
    location_box = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[3]/div[2]/div[1]/div/div[1]')
    location_box.click()
    time.sleep(random.uniform(2, 4))
    location_input = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[3]/div[2]/div[1]/div/div[1]/div/div/div/input')
    location_input.send_keys('Belgium')
    belgium = driver.find_element_by_xpath('//*[@id="option_N,25"]/span')
    belgium.click()
    return driver, driver.current_url

In [202]:
def get_lux_reviews(driver):
    time.sleep(random.uniform(2, 4))
    filter_button = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[2]/div[2]/button/span')
    filter_button.click()
    time.sleep(random.uniform(2, 4))
    location_box = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[3]/div[2]/div[1]/div/div[1]')
    location_box.click()
    time.sleep(random.uniform(2, 4))
    location_input = driver.find_element_by_xpath('//*[@id="MainContent"]/div/div[1]/div[1]/div[1]/div/div[1]/div[3]/div[2]/div[1]/div/div[1]/div/div/div/input')
    location_input.send_keys('Luxembourg')
    luxembourg = driver.find_element_by_xpath('//*[@id="option_S,3840"]/span')
    luxembourg.click()
    return driver, driver.current_url

In [33]:
# Set site language to us
def set_site_language(driver):
    time.sleep(random.uniform(2, 4))
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(random.uniform(2, 4))
    language_choice = driver.find_element_by_xpath('//*[@id="Footer"]/nav/ul[2]/li[3]/div/div/div[1]')
    language_choice.click()
    soup = BeautifulSoup(driver.page_source, 'lxml')
    language_click_box = soup.find('div', class_='dropdownOptions dropdownExpanded animated above')
    for li in language_click_box.find_all('li'):
        country = li.find_all('span')[1].string
        if country == 'United States':
            us_id = li.get('id')
            break
    time.sleep(random.uniform(2, 4))
    us = driver.find_element_by_id(us_id)
    us.click()
    time.sleep(random.uniform(2, 4))
    return driver

In [197]:
# Main scraping function
def getting_company_reviews_urls_bs(companies):
    """
    Takes a list of company names as input and returns the url of the review 
    section of each company in a dataframe
    """
    search_bar_xpath = '//*[@id="KeywordSearch"]'
    df = pd.DataFrame({'company': [], 'belgian_reviews_link': [], 'lux_reviews_link': []})
    #domain = 'https://fr.glassdoor.be'

    
    for company in companies:
         
        #Setting up driver    
        driver = setup_driver_to_reviews_search(url)
        
        #Searching for the company
        time.sleep(random.uniform(4, 6))
        driver = set_site_language(driver)
        time.sleep(random.uniform(2, 4))
        company_search = driver.find_element_by_xpath(search_bar_xpath)
        company_search.send_keys(company)
        time.sleep(random.uniform(1, 2))
        company_search.send_keys(Keys.ENTER)
        print(f'Found {company}')
        
        try:
            #Going on the first company's page
            time.sleep(random.uniform(2, 4))
            driver.find_element_by_xpath('//*[@id="MainCol"]/div/div[1]/div/div[1]/div/div[2]/h2/a').click()
            print(f'Clicked on first link for {company}')

            #Going in the reviews
            #### FAUT TROUVER NOUVEAU MOYEN D'AVOIR LES REVIEWS
            time.sleep(random.uniform(2, 4))
            try:
                reviews_button = driver.find_element_by_xpath('//*[@id="EIOverviewContainer"]/div/div[5]/div[3]/a')
                desired_y = reviews_button.location['y'] - 200
                driver.execute_script("window.scrollBy(0, arguments[0]);", desired_y)
                time.sleep(random.uniform(2, 4))
                reviews_button.click()
            except (NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException, WebDriverException):
                get_rid_overlay(driver)
                time.sleep(random.uniform(2, 4))
                reviews_button = driver.find_element_by_xpath('//*[@id="EIOverviewContainer"]/div/div[5]/div[3]/a')
                desired_y = reviews_button.location['y'] - 2
                time.sleep(random.uniform(2, 4))
                driver.execute_script("window.scrollBy(0, arguments[0]);", desired_y)
                time.sleep(random.uniform(2, 4))
                reviews_button.click()
            print(f'Found reviews for {company}')


            #Get belgian reviews link
            time.sleep(random.uniform(2, 4))
            try: 
                driver, belgian_reviews = get_belgian_reviews(driver)
            except (ElementClickInterceptedException, ElementNotInteractableException, WebDriverException):
                get_rid_overlay(driver)
                time.sleep(random.uniform(2, 4))
                driver, belgian_reviews = get_belgian_reviews(driver)
            print(belgian_reviews)

            #Get Luxembourg reviews link
            time.sleep(random.uniform(2, 4))
            try: 
                driver, lux_reviews = get_lux_reviews(driver)
            except (ElementClickInterceptedException, ElementNotInteractableException, WebDriverException):
                get_rid_overlay(driver)
                time.sleep(random.uniform(2, 4))
                driver, lux_reviews = get_lux_reviews(driver)
            print(lux_reviews)

            #Populating the dataframe
            df = df.append({'company': company,
                            'belgian_reviews_link': belgian_reviews,
                           'lux_reviews_link': lux_reviews},
                           ignore_index=True)
            return df
            except: 
                continue
        driver.close()
        
    
    #df.to_csv('data/company_links.csv')